In [723]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

os.chdir(r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\Data Cleaning\22 July 2025\1992")
os.getcwd()

# Filepaths for the datasets
filepaths = {
    'df1': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\Data Cleaning\1992_datasets\Blocks 1,2,3,4,5_Visit 1_Household characteristics.dta",
    'df2': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\Data Cleaning\1992_datasets\Blocks 14,15,16_Visit 1_Ownership of shares etc.dta",
    'df3': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\Data Cleaning\1992_datasets\Block 17pt2_Visit 1_Particulars of cash loans payable by the household and transactions of loans.dta",
    'df4': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\Data Cleaning\1992_datasets\Block 10_Visit 1_Owned buildings and other constructions and their transactions.dta",
    'df5': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\Data Cleaning\1992_datasets\Block 7_Visit 1_Area of land owned as on date of survey and transactions.dta",
}


In [724]:
# Load dataframes
dataframes = {name: pd.read_stata(path) for name, path in filepaths.items()}

# Print unique HHIDs in first seven datasets
for i in range(1, 6):
    df = dataframes[f'df{i}']
    unique_hhids = set(df['HHID'].dropna())
    print(f"Unique HHIDs in df{i}: {len(unique_hhids)}")

# Compute common HHIDs across the first five datasets
common_hhids = set.intersection(*[
    set(dataframes[f'df{i}']['HHID'].dropna()) for i in range(1, 6)
]) #& set.union(*[
    #set(dataframes[f'df{i}']['HHID'].dropna()) for i in range(6, 6)
#])
print(f"Common HHIDs across: {len(common_hhids)}")

# Filter each dataframe to keep only rows with HHID in common_hhids
for name in dataframes:
    df = dataframes[name]
    df_filtered = df[df['HHID'].isin(common_hhids)].copy()
    dataframes[name] = df_filtered  # Replace with filtered version

# Optional: Print how many rows remain in each
for name, df in dataframes.items():
    print(f"{name}: {len(df)} rows after filtering")

# Reset index
for key in dataframes:
    dataframes[key] = dataframes[key].reset_index(drop=True)

# Convert all columns to numeric except 'HHID' for each dataframe
for i in range(1, 6):
    df = dataframes[f'df{i}']
    cols_to_convert = df.columns.difference(['HHID'])
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
    dataframes[f'df{i}'] = df  # Update dictionary

# Unpack if needed
df1, df2, df3, df4, df5 = [dataframes[f'df{i}'] for i in range(1, 6)]


Unique HHIDs in df1: 57031
Unique HHIDs in df2: 46458
Unique HHIDs in df3: 25133
Unique HHIDs in df4: 47174
Unique HHIDs in df5: 46548
Common HHIDs across: 16893
df1: 16893 rows after filtering
df2: 52501 rows after filtering
df3: 23622 rows after filtering
df4: 83797 rows after filtering
df5: 61175 rows after filtering


In [725]:
df2['Block_No'].value_counts(dropna=False)
#df2['Block_No'].dtype

Block_No
15    44081
14     6611
16     1809
Name: count, dtype: int64

In [726]:
# Get unique HHIDs for Block_No 15 and 14
hhid_15 = set(df2.loc[df2['Block_No'] == 15, 'HHID'].unique())
hhid_14 = set(df2.loc[df2['Block_No'] == 14, 'HHID'].unique())

# Find common HHIDs
common_hhids1 = hhid_15.intersection(hhid_14)

# Print result
print(f"✅ Number of common unique HHIDs between Block_No 15 and 14: {len(common_hhids1)}")


✅ Number of common unique HHIDs between Block_No 15 and 14: 2899


In [727]:
len(hhid_15)

16628

In [728]:
print(f'Unique HHIDs in Shares and Debentures: {len(hhid_14)}')
print(f'Unique HHIDs in Fin Assets (excl. shares and deb): {len(hhid_15)}')

Unique HHIDs in Shares and Debentures: 3118
Unique HHIDs in Fin Assets (excl. shares and deb): 16628


In [729]:
# Filter each dataframe to keep only rows with HHID in common_hhids1
for name in dataframes:
    df = dataframes[name]
    df_filtered = df[df['HHID'].isin(common_hhids1)].copy()
    dataframes[name] = df_filtered  # Replace with filtered version

# Optional: Print how many rows remain in each
for name, df in dataframes.items():
    print(f"{name}: {len(df)} rows after filtering")

# Reset index
for key in dataframes:
    dataframes[key] = dataframes[key].reset_index(drop=True)

# Convert all columns to numeric except 'HHID' for each dataframe
for i in range(1, 6):
    df = dataframes[f'df{i}']
    cols_to_convert = df.columns.difference(['HHID'])
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
    dataframes[f'df{i}'] = df  # Update dictionary

# Unpack if needed
df1, df2, df3, df4, df5 = [dataframes[f'df{i}'] for i in range(1, 6)]


df1: 2899 rows after filtering
df2: 15991 rows after filtering
df3: 4893 rows after filtering
df4: 14933 rows after filtering
df5: 11410 rows after filtering


In [730]:
print(df2.columns)

Index(['HHID', 'RoundSchedule', 'Sample', 'Visit_no', 'State_Region', 'State',
       'FlotNo', 'Sector', 'Stratum', 'SubSample', 'SubRound', 'Vill_Blk_Slno',
       'SubStratum', 'Hhold_no', 'Level', 'Block_No', 'B14_15_16_q1',
       'Sign_of_next_field', 'B14_15_16_q3', 'B14_15_16_q4', 'B14_15_16_q5',
       'Sign_of_next_field2', 'B14_15_16_q6', 'Update_Code'],
      dtype='object')


In [731]:
# Keeping relevant variables and merging

# Initial row count
initial_rows = len(df1)

# Keep only rows where B5_q16 is not missing
df1 = df1[df1['B5_q16'].notna()]
df1 = df1[df1['HHID'].notna()]

# Final row count
final_rows = len(df1)
dropped_rows = initial_rows - final_rows

# Print summary
print(f"✅ HHIDs retained: {final_rows}")
print(f"❌ HHIDs dropped (missing B5_q16): {dropped_rows}")



✅ HHIDs retained: 2897
❌ HHIDs dropped (missing B5_q16): 2


In [732]:
df1_hhid = set(df1['HHID'])

In [733]:
print(len(df1_hhid))

2897


In [734]:
for name in dataframes:
    df = dataframes[name]
    df_filtered = df[df['HHID'].isin(df1_hhid)].copy()
    dataframes[name] = df_filtered  # Replace with filtered version

# Optional: Print how many rows remain in each
for name, df in dataframes.items():
    print(f"{name}: {len(df)} rows after filtering")

# Reset index
for key in dataframes:
    dataframes[key] = dataframes[key].reset_index(drop=True)

# Convert all columns to numeric except 'HHID' for each dataframe
for i in range(1, 6):
    df = dataframes[f'df{i}']
    cols_to_convert = df.columns.difference(['HHID'])
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
    dataframes[f'df{i}'] = df  # Update dictionary

# Unpack if needed
df1, df2, df3, df4, df5 = [dataframes[f'df{i}'] for i in range(1, 6)]


df1: 2897 rows after filtering
df2: 15980 rows after filtering
df3: 4890 rows after filtering
df4: 14923 rows after filtering
df5: 11393 rows after filtering


In [735]:
df1.to_stata("df1_1992.dta", write_index=False)
df2.to_stata("df2_1992.dta", write_index=False)
df3.to_stata("df3_1992.dta", write_index=False)
df4.to_stata("df4_1992.dta", write_index=False)
df5.to_stata("df5_1992.dta", write_index=False)

In [736]:
print(df2.columns)

Index(['HHID', 'RoundSchedule', 'Sample', 'Visit_no', 'State_Region', 'State',
       'FlotNo', 'Sector', 'Stratum', 'SubSample', 'SubRound', 'Vill_Blk_Slno',
       'SubStratum', 'Hhold_no', 'Level', 'Block_No', 'B14_15_16_q1',
       'Sign_of_next_field', 'B14_15_16_q3', 'B14_15_16_q4', 'B14_15_16_q5',
       'Sign_of_next_field2', 'B14_15_16_q6', 'Update_Code'],
      dtype='object')


In [737]:
# df1
df1 = df1[['HHID', 'State', 'B5_q16']].copy()

In [738]:
# df2

# Filter df2 for Block_No 14 or 15 and select desired columns
df2 = df2[df2['Block_No'].isin([14, 15])][['HHID', 'Block_No', 'B14_15_16_q1', 'B14_15_16_q3', 'B14_15_16_q6']].copy()       # Fin Assets AND Shares, Debentures

hhid2 = set(df2['HHID'].dropna())
print(f'Unique HHIDs in df2: {len(hhid2)}')

# df2 already contains only Block_No 14 and 15
# Split into two based on Block_No
df2_1 = df2[df2['Block_No'] == 14].copy()                    # Shares
df2_1 = df2_1[df2_1['B14_15_16_q1'].isin([8])]

df2_2 = df2[df2['Block_No'] == 15].copy()                    # Fin Assets
df2_2 = df2_2[df2_2['B14_15_16_q1'].isin([1,2,3,4,5,6,9,10,11,12,13,14])]

hhid2_1 = set(df2_1['HHID'].dropna())
print(f'Unique HHIDs in df2_1: {len(hhid2_1)}')

hhid2_2 = set(df2_2['HHID'].dropna())
print(f'Unique HHIDs in df2_2: {len(hhid2_2)}')

Unique HHIDs in df2: 2897
Unique HHIDs in df2_1: 2897
Unique HHIDs in df2_2: 2886


In [739]:
# df3
df3 = df3[['HHID', 'B17_2_q1', 'B17_2_q6', 'B17_2_q24', 'B17_2_q25']].copy()         # Loan

In [740]:
# df3
df3 = df3[df3['B17_2_q6'].isin([1,2,3,4,5,6])]

In [741]:
print(df4.columns)

Index(['HHID', 'RoundSchedule', 'Sample', 'Visit_no', 'State_Region', 'State',
       'FlotNo', 'Sector', 'Stratum', 'SubSample', 'SubRound', 'Vill_Blk_Slno',
       'SubStratum', 'Hhold_no', 'Level', 'B10_q1', 'Record_No', 'B10_q3_q13',
       'B10_q4_q14', 'B10_q5_q15', 'B10_q6_q16', 'B10_q7_q17', 'B10_q8_q18',
       'B10_q9_q19', 'B10_q10_q20', 'B10_q11_q21', 'B10_q12_q22',
       'Update_Code'],
      dtype='object')


In [742]:
df4['B10_q12_q22'].isna().sum()

0

In [743]:
df4['B10_q1'].dtype

dtype('int64')

In [744]:
# df4
df4 = df4[['HHID', 'B10_q1', 'B10_q12_q22']].copy()         # Buildings- Block 10

df4 = df4[df4['B10_q1'] == 11] 

In [745]:
print(df5.columns)

Index(['HHID', 'RoundSchedule', 'Sample', 'Visit_no', 'State_Region', 'State',
       'FlotNo', 'Sector', 'Stratum', 'SubSample', 'SubRound', 'Vill_Blk_Slno',
       'SubStratum', 'Hhold_no', 'Level', 'B7_q1', 'B7_q3', 'B7_q4', 'B7_q5',
       'B7_q6', 'B7_q7', 'B7_q8', 'B7_q9', 'B7_q10', 'B7_q11', 'B7_q12',
       'B7_q13', 'Update_Code'],
      dtype='object')


In [746]:
# df5
df5 = df5[['HHID', 'B7_q1', 'B7_q7', 'B7_q13']].copy()     # Land- Block 5

df5 = df5[df5['B7_q1'] == 99] 

In [747]:
# Check for missing vals in MCE
df1['B5_q16'].isna().sum()

0

In [748]:
print(df2_1.columns)

Index(['HHID', 'Block_No', 'B14_15_16_q1', 'B14_15_16_q3', 'B14_15_16_q6'], dtype='object')


In [749]:
print(df2_1['Block_No'].value_counts(dropna=False))             # df2_1 is block 14- Shares and Debentures
print(df2_2['Block_No'].value_counts(dropna=False))             # df2_2 is block 15- Fin Assets (excl. shares)

Block_No
14    2897
Name: count, dtype: int64
Block_No
15    6112
Name: count, dtype: int64


In [750]:
df2_1.drop(columns=['Block_No'], inplace= True)
df2_2.drop(columns=['Block_No'], inplace= True)

In [751]:
print(df1.columns)
print(df2_1.columns)
print(df2_2.columns)
print(df3.columns)
print(df4.columns)
print(df5.columns)

Index(['HHID', 'State', 'B5_q16'], dtype='object')
Index(['HHID', 'B14_15_16_q1', 'B14_15_16_q3', 'B14_15_16_q6'], dtype='object')
Index(['HHID', 'B14_15_16_q1', 'B14_15_16_q3', 'B14_15_16_q6'], dtype='object')
Index(['HHID', 'B17_2_q1', 'B17_2_q6', 'B17_2_q24', 'B17_2_q25'], dtype='object')
Index(['HHID', 'B10_q1', 'B10_q12_q22'], dtype='object')
Index(['HHID', 'B7_q1', 'B7_q7', 'B7_q13'], dtype='object')


In [752]:
'''

'df1': MCE,

'df2_1': Shares and Debentures,                         # rename df3

'df2_2': Fin Assets excluding shares,                   # rename df2

'df3': Loan,                                            # rename df4

'df4': Buildings,                                       # rename df5

'df5': Land                                             # rename df6

'''

"\n\n'df1': MCE,\n\n'df2_1': Shares and Debentures,                         # rename df3\n\n'df2_2': Fin Assets excluding shares,                   # rename df2\n\n'df3': Loan,                                            # rename df4\n\n'df4': Buildings,                                       # rename df5\n\n'df5': Land                                             # rename df6\n\n"

In [753]:
df6 = df5.copy()
df5 = df4.copy()
df4 = df3.copy()
df3 = df2_1.copy()
df2 = df2_2.copy()

In [754]:
df1.to_stata("df1_1992.dta", write_index=False)
df2.to_stata("df2_1992.dta", write_index=False)
df3.to_stata("df3_1992.dta", write_index=False)
df4.to_stata("df4_1992.dta", write_index=False)
df5.to_stata("df5_1992.dta", write_index=False)
df6.to_stata("df6_1992.dta", write_index=False)

In [755]:
#### df5

# Filter only rows where B10_q1 == 11
df_filtered = df5[df5['B10_q1'] == 11]

# Count entries per household
counts = df_filtered.groupby('HHID').size()

# Find households with more than 1 entry
multi_entry_households = counts[counts > 1]

# Number of such households
num_households = multi_entry_households.shape[0]

print("Number of households with >1 entries in B10_q1:", num_households)


Number of households with >1 entries in B10_q1: 2896


In [756]:
# Filter rows where B10_q1 == 11
df_filtered = df5[df5['B10_q1'] == 11]

# Keep only households with more than 1 row
multi_entry_households = df_filtered.groupby('HHID').filter(lambda x: len(x) > 1)

# Group by HHID and check if all rows have non-zero B10_q12_q22
valid_households = multi_entry_households.groupby('HHID').filter(
    lambda x: (x['B10_q12_q22'] != 0).all()
)

# Count number of such households
num_valid_households = valid_households['HHID'].nunique()
print("Number of households with >1 rows and non-zero in all rows:", num_valid_households)


Number of households with >1 rows and non-zero in all rows: 212


In [757]:
import pandas as pd

# --- Step 1: Filter for B10_q1 == 11 ---
df_filtered = df5[df5['B10_q1'] == 11]

# --- Step 2: Keep only households with more than 1 row ---
multi_entry_households = df_filtered.groupby('HHID').filter(lambda x: len(x) > 1)

# --- Step 3: Keep only households where all rows have non-zero B10_q12_q22 ---
valid_households = multi_entry_households.groupby('HHID').filter(
    lambda x: (x['B10_q12_q22'] != 0).all()
)

# --- Step 4: Sort for readability ---
valid_households = valid_households.sort_values(by=['HHID'])

# --- Step 5: Export to CSV ---
valid_households.to_csv("households_nonzero_B10_q12_q22.csv", index=False)

print("✅ Exported 212 households to 'households_nonzero_B10_q12_q22.csv'")


✅ Exported 212 households to 'households_nonzero_B10_q12_q22.csv'


In [758]:
# Filter only rows where B10_q1 == 11
df_filtered = df5[df5['B10_q1'] == 11]

# Count entries per household
counts = df_filtered.groupby('HHID').size()

# Get household(s) with exactly 1 entry
single_entry_households = counts[counts == 1].index

# Show their rows
single_entry_rows = df_filtered[df_filtered['HHID'].isin(single_entry_households)]
print(single_entry_rows)


          HHID  B10_q1  B10_q12_q22
11842  4123461      11       225000


In [759]:
import pandas as pd

# One-liner: filter, then pick max per household (works for 1-row households too)
final_rows = (
    df5.loc[df5['B10_q1'] == 11]
       .loc[lambda d: d.groupby('HHID')['B10_q12_q22'].idxmax()]
       .sort_values('HHID')
)

# Export to Stata
final_rows.to_stata('yupp.dta', write_index=False)

print("✅ Exported all 2899 households (max per HHID) to 'yupp.dta'")


✅ Exported all 2899 households (max per HHID) to 'yupp.dta'


In [760]:
df5 = final_rows.copy()

In [761]:
df1.to_stata("df1_1992.dta", write_index=False)
df2.to_stata("df2_1992.dta", write_index=False)
df3.to_stata("df3_1992.dta", write_index=False)
df4.to_stata("df4_1992.dta", write_index=False)
df5.to_stata("df5_1992.dta", write_index=False)
df6.to_stata("df6_1992.dta", write_index=False)

In [762]:
print(df1.columns)
print(df2.columns)
print(df3.columns)
print(df4.columns)
print(df5.columns)
print(df6.columns)

Index(['HHID', 'State', 'B5_q16'], dtype='object')
Index(['HHID', 'B14_15_16_q1', 'B14_15_16_q3', 'B14_15_16_q6'], dtype='object')
Index(['HHID', 'B14_15_16_q1', 'B14_15_16_q3', 'B14_15_16_q6'], dtype='object')
Index(['HHID', 'B17_2_q1', 'B17_2_q6', 'B17_2_q24', 'B17_2_q25'], dtype='object')
Index(['HHID', 'B10_q1', 'B10_q12_q22'], dtype='object')
Index(['HHID', 'B7_q1', 'B7_q7', 'B7_q13'], dtype='object')


In [763]:
print(df3['B14_15_16_q1'].value_counts(dropna=False))             # df3 is block 14- Shares and Debentures
print(df2['B14_15_16_q1'].value_counts(dropna=False))             # df2 is block 15- Fin Assets (excl. shares)

B14_15_16_q1
8    2897
Name: count, dtype: int64
B14_15_16_q1
14    2750
5      795
9      674
10     657
4      336
1      255
3      236
6      132
2      118
12      93
13      58
11       8
Name: count, dtype: int64


In [764]:
print(df1.columns)
print(df2.columns)
print(df3.columns)
print(df4.columns)
print(df5.columns)
print(df6.columns)

Index(['HHID', 'State', 'B5_q16'], dtype='object')
Index(['HHID', 'B14_15_16_q1', 'B14_15_16_q3', 'B14_15_16_q6'], dtype='object')
Index(['HHID', 'B14_15_16_q1', 'B14_15_16_q3', 'B14_15_16_q6'], dtype='object')
Index(['HHID', 'B17_2_q1', 'B17_2_q6', 'B17_2_q24', 'B17_2_q25'], dtype='object')
Index(['HHID', 'B10_q1', 'B10_q12_q22'], dtype='object')
Index(['HHID', 'B7_q1', 'B7_q7', 'B7_q13'], dtype='object')


In [765]:
# Renaming, creating, dropping columns 

df1  = df1.rename(columns={
    'B5_q16': 'MCE'
})

df2  = df2.rename(columns={
    'B14_15_16_q1': 'Fin_Asset_Serial',
    'B14_15_16_q3': 'Fin_Asset_val1',
    'B14_15_16_q6': 'Fin_Asset_val2'
})

df3  = df3.rename(columns={
    'B14_15_16_q1': 'Share_Asset_serial',
    'B14_15_16_q3': 'Share_Asset_val1',
    'B14_15_16_q6': 'Share_Asset_val2'
})

df4  = df4.rename(columns={
    'B17_2_q1': 'Liability_serial',
    'B17_2_q6': 'Credit_Agency',
    'B17_2_q24': 'Liability_val1',
    'B17_2_q25': 'Liability_val2'
})


df5  = df5.rename(columns={
    'B10_q1': 'Building_serial',
    'B10_q12_q22': 'Building_value'
})


df6  = df6.rename(columns={
    'B7_q1': 'Land_serial',
    'B7_q7': 'Land_val1',
    'B7_q13': 'Land_val2'
})

In [766]:
df1.to_stata("df1_1992.dta", write_index=False)
df2.to_stata("df2_1992.dta", write_index=False)
df3.to_stata("df3_1992.dta", write_index=False)
df4.to_stata("df4_1992.dta", write_index=False)
df5.to_stata("df5_1992.dta", write_index=False)
df6.to_stata("df6_1992.dta", write_index=False)

In [767]:
print(df1.columns)
print(df2.columns)
print(df3.columns)
print(df4.columns)
print(df5.columns)
print(df6.columns)

Index(['HHID', 'State', 'MCE'], dtype='object')
Index(['HHID', 'Fin_Asset_Serial', 'Fin_Asset_val1', 'Fin_Asset_val2'], dtype='object')
Index(['HHID', 'Share_Asset_serial', 'Share_Asset_val1', 'Share_Asset_val2'], dtype='object')
Index(['HHID', 'Liability_serial', 'Credit_Agency', 'Liability_val1',
       'Liability_val2'],
      dtype='object')
Index(['HHID', 'Building_serial', 'Building_value'], dtype='object')
Index(['HHID', 'Land_serial', 'Land_val1', 'Land_val2'], dtype='object')


In [768]:
df6['Land_serial'].value_counts()

Land_serial
99    2897
Name: count, dtype: int64

In [769]:
# df2
#df2 = df2[df2['Fin_Asset_Serial'].isin([1,2,3,4,5,6,7,8,9,10,11,12,13,14])]

# df3
#df3 = df3[df3['Share_Asset_serial'] == 8] 

# df4
#df4 = df4[df4['Credit_Agency'].isin([1,2,3,4,5,6])]

In [770]:
df2['Fin_Asset_Serial'].value_counts()

Fin_Asset_Serial
14    2750
5      795
9      674
10     657
4      336
1      255
3      236
6      132
2      118
12      93
13      58
11       8
Name: count, dtype: int64

In [771]:
df3['Share_Asset_serial'].value_counts()

Share_Asset_serial
8    2897
Name: count, dtype: int64

In [772]:
df4['Credit_Agency'].value_counts()

Credit_Agency
2.0    2064
3.0     910
1.0     316
5.0     152
6.0     110
4.0      51
Name: count, dtype: int64

In [773]:
# Check for missing values in df1

cols = ['HHID', 'State', 'MCE']

# Create summary dataframe
summary = pd.DataFrame({
    'Column': cols,
    'Missing_Values': [df1[col].isna().sum() for col in cols],
    'Zero_Values': [(df1[col] == 0).sum() for col in cols]
})

print(summary)


# Unique HHIDs

unique_hhids = set(df1['HHID'].dropna())
print(f"Unique HHIDs: {len(unique_hhids)}")

  Column  Missing_Values  Zero_Values
0   HHID               0            0
1  State               0            0
2    MCE               0            0
Unique HHIDs: 2897


In [774]:
df2.columns

Index(['HHID', 'Fin_Asset_Serial', 'Fin_Asset_val1', 'Fin_Asset_val2'], dtype='object')

In [775]:
# df2
df2['Fin_Asset_Value'] = np.where(
    df2['Fin_Asset_Serial'] == 14,
    df2['Fin_Asset_val1'],
    df2['Fin_Asset_val1'].combine_first(df2['Fin_Asset_val2'])
)
df2['Fin_Asset_Value'] = pd.Series(df2['Fin_Asset_Value']).fillna(0)

# df3 (Share Asset value)
df3['Share_Asset_Value'] = df3['Share_Asset_val1'].combine_first(df3['Share_Asset_val2'])
df3['Share_Asset_Value'] = df3['Share_Asset_Value'].fillna(0)
df3 = df3.reset_index(drop=True)

# Mapping dictionary using float keys (since your column is float64)
Share_Asset_Map = {
    8.0: 'Shares and Debentures'
}

# df4 (Loans)
df4['Liability_value'] = df4['Liability_val1'].combine_first(df4['Liability_val2'])
df4['Liability_value'] = df4['Liability_value'].fillna(0)

# df5 (Buildings value)
df5['Building_value'] = df5['Building_value'].fillna(0)

# df6 (Land value)

df6['Land_value'] = df6['Land_val1'].combine_first(df6['Land_val2'])
df6['Land_value'] = df6['Land_value'].fillna(0)

C:\Users\siddu\AppData\Local\Temp\ipykernel_39844\3678178701.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df6['Land_value'] = df6['Land_val1'].combine_first(df6['Land_val2'])


In [776]:
df4['Credit_Agency'].value_counts()

Credit_Agency
2.0    2064
3.0     910
1.0     316
5.0     152
6.0     110
4.0      51
Name: count, dtype: int64

In [777]:
df5['Building_value'].value_counts(dropna=False)

Building_value
50000      97
40000      89
100000     85
30000      78
20000      77
           ..
16600       1
30700       1
134670      1
197650      1
5000000     1
Name: count, Length: 605, dtype: int64

In [778]:
df2['Fin_Asset_Serial'].value_counts()

Fin_Asset_Serial
14    2750
5      795
9      674
10     657
4      336
1      255
3      236
6      132
2      118
12      93
13      58
11       8
Name: count, dtype: int64

In [779]:
print(df2['Fin_Asset_Serial'].value_counts(dropna=False))

# Step 2: Group by HHID and sum Fin_Asset_Value
summary_rows = df2.groupby('HHID', as_index=False)['Fin_Asset_Value'].sum()
summary_rows['Fin_Asset_Serial'] = 999

# Step 3: Add other columns with NaN or defaults
for col in df2.columns:
    if col not in ['HHID', 'Fin_Asset_Serial', 'Fin_Asset_Value']:
        summary_rows[col] = np.nan

# Step 4: Concatenate the new rows
df2_mod = pd.concat([df2, summary_rows], ignore_index=True)

# Step 5: Sort
df2_mod = df2_mod.sort_values(by=['HHID', 'Fin_Asset_Serial']).reset_index(drop=True)

# Step 6: Save
df2_mod.to_stata("mergedforu.dta", write_index=False)

print(df2_mod.head())
print(df2_mod['HHID'].nunique())

df2_mod['Fin_Asset_Serial'].value_counts(dropna=False)

# Modify serials and labels, if any

# df2_mod
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([14]), 'Fin_Asset_Serial'] = 31
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([4,5,6]), 'Fin_Asset_Serial'] = 32
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([9]), 'Fin_Asset_Serial'] = 34
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([10,11]), 'Fin_Asset_Serial'] = 35
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([1,2,3,12,13]), 'Fin_Asset_Serial'] = 36
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([999]), 'Fin_Asset_Serial'] = 37

df2_mod = df2_mod.reset_index(drop=True)

# Fin Asset Mapping

Fin_Asset_map = {

    31.0: 'Currency',
    32.0: 'Deposits',
    34.0: 'Life Insurance fund',
    35.0: 'Provident and Pension fund',
    36.0: 'Claims on Government',
    37.0: 'Total financial assets (other thanshares and related instruments)'
}

df2_mod = df2_mod.groupby(['HHID', 'Fin_Asset_Serial'], as_index=False).agg({
    'Fin_Asset_Value': 'sum'  
})

df2_mod = df2_mod.reset_index(drop=True)

df2_mod.to_stata("mergedforu.dta", write_index=False)

df2_mod['Fin_Asset_Serial'].value_counts()

df2 = df2_mod.copy()

Fin_Asset_Serial
14    2750
5      795
9      674
10     657
4      336
1      255
3      236
6      132
2      118
12      93
13      58
11       8
Name: count, dtype: int64


      HHID  Fin_Asset_Serial  Fin_Asset_val1  Fin_Asset_val2  Fin_Asset_Value
0  3104731                14           380.0             NaN            380.0
1  3104731               999             NaN             NaN            380.0
2  3107711                14           150.0             NaN            150.0
3  3107711               999             NaN             NaN            150.0
4  3107712                14           120.0             NaN            120.0
2886


In [780]:
df4.columns

Index(['HHID', 'Liability_serial', 'Credit_Agency', 'Liability_val1',
       'Liability_val2', 'Liability_value'],
      dtype='object')

In [781]:
print(df4['Credit_Agency'].value_counts(dropna=False))

# Step 2: Group by HHID and sum Liability_value
summary_rows_liab = df4.groupby('HHID', as_index=False)['Liability_value'].sum()
summary_rows_liab['Credit_Agency'] = 999

# Step 3: Add other columns with NaN or defaults
for col in df4.columns:
    if col not in ['HHID', 'Credit_Agency', 'Liability_value']:
        summary_rows_liab[col] = np.nan

# Step 4: Concatenate the new rows
df4_mod = pd.concat([df4, summary_rows_liab], ignore_index=True)

# Step 5: Sort
df4_mod = df4_mod.sort_values(by=['HHID', 'Credit_Agency']).reset_index(drop=True)

# Step 6: Save
df4_mod.to_stata("mergedforu_liab.dta", write_index=False)

print(df4_mod.head())
print(df4_mod['HHID'].nunique())


# df4_mod
df4_mod.loc[df4_mod['Credit_Agency'].isin([2,3]), 'Credit_Agency'] = 91
df4_mod.loc[df4_mod['Credit_Agency'].isin([1,4,5,6]), 'Credit_Agency'] = 92

df4_mod = df4_mod.reset_index(drop=True)

# Liability Map

Credit_Agency_Map = {

    91.0: 'Bank advances',
    92.0: 'Non-banking loans and advances',
    999.0: 'Total Institutional Liabilities'
}

df4_mod = df4_mod.groupby(['HHID', 'Credit_Agency'], as_index=False).agg({
    'Liability_value': 'sum'  
})

df4_mod = df4_mod.reset_index(drop=True)

df4_mod.to_stata("mergedforu_liab.dta", write_index=False)

df4_mod['Credit_Agency'].value_counts()

df4 = df4_mod.copy()

Credit_Agency
2.0    2064
3.0     910
1.0     316
5.0     152
6.0     110
4.0      51
Name: count, dtype: int64


      HHID  Liability_serial  Credit_Agency  Liability_val1  Liability_val2  \
0  3104731               1.0            2.0             NaN             NaN   
1  3104731               NaN          999.0             NaN             NaN   
2  3107711               1.0            3.0             NaN             NaN   
3  3107711               NaN          999.0             NaN             NaN   
4  3107712               1.0            3.0             NaN             NaN   

   Liability_value  
0              0.0  
1              0.0  
2              0.0  
3              0.0  
4              0.0  
2487


In [782]:
print(df1.columns)
print(df2.columns)
print(df3.columns)
print(df4.columns)
print(df5.columns)
print(df6.columns)

Index(['HHID', 'State', 'MCE'], dtype='object')
Index(['HHID', 'Fin_Asset_Serial', 'Fin_Asset_Value'], dtype='object')
Index(['HHID', 'Share_Asset_serial', 'Share_Asset_val1', 'Share_Asset_val2',
       'Share_Asset_Value'],
      dtype='object')
Index(['HHID', 'Credit_Agency', 'Liability_value'], dtype='object')
Index(['HHID', 'Building_serial', 'Building_value'], dtype='object')
Index(['HHID', 'Land_serial', 'Land_val1', 'Land_val2', 'Land_value'], dtype='object')


In [783]:
# Dropping unnecessary columns

df3.drop(columns=['Share_Asset_val1', 'Share_Asset_val2'], inplace=True)
df6.drop(columns=['Land_val1', 'Land_val2'], inplace=True)

In [784]:
print(df1.columns)
print(df2.columns)
print(df3.columns)
print(df4.columns)
print(df5.columns)
print(df6.columns)

Index(['HHID', 'State', 'MCE'], dtype='object')
Index(['HHID', 'Fin_Asset_Serial', 'Fin_Asset_Value'], dtype='object')
Index(['HHID', 'Share_Asset_serial', 'Share_Asset_Value'], dtype='object')
Index(['HHID', 'Credit_Agency', 'Liability_value'], dtype='object')
Index(['HHID', 'Building_serial', 'Building_value'], dtype='object')
Index(['HHID', 'Land_serial', 'Land_value'], dtype='object')


In [785]:
df1.to_stata("df1_1992.dta", write_index=False)
df2.to_stata("df2_1992.dta", write_index=False)
df3.to_stata("df3_1992.dta", write_index=False)
df4.to_stata("df4_1992.dta", write_index=False)
df5.to_stata("df5_1992.dta", write_index=False)
df6.to_stata("df6_1992.dta", write_index=False)

In [786]:
dataframes['df1'] = df1
dataframes['df2'] = df2
dataframes['df3'] = df3
dataframes['df4'] = df4
dataframes['df5'] = df5
dataframes['df6'] = df6

In [787]:
# Print unique HHIDs in first seven datasets
for i in range(1, 7):
    df = dataframes[f'df{i}']
    unique_hhids = set(df['HHID'].dropna())
    print(f"Unique HHIDs in df{i}: {len(unique_hhids)}")

# Compute common HHIDs: intersection of first five & union of df6, df7
common_hhids = set.intersection(*[
    set(dataframes[f'df{i}']['HHID'].dropna()) for i in range(1, 7)
])

print(f"Common HHIDs across: {len(common_hhids)}")

# Filter each dataframe to keep only rows with HHID in common_hhids
for name in dataframes:
    df = dataframes[name]
    df_filtered = df[df['HHID'].isin(common_hhids)].copy()
    dataframes[name] = df_filtered  # Replace with filtered version

# Optional: Print how many rows remain in each
for name, df in dataframes.items():
    print(f"{name}: {len(df)} rows after filtering")

# Reset index
for key in dataframes:
    dataframes[key] = dataframes[key].reset_index(drop=True)

# Convert all columns to numeric except 'HHID' for each dataframe
for i in range(1, 7):
    df = dataframes[f'df{i}']
    cols_to_convert = df.columns.difference(['HHID'])
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
    dataframes[f'df{i}'] = df  # Update dictionary

# Unpack if needed
df1, df2, df3, df4, df5, df6 = [dataframes[f'df{i}'] for i in range(1, 7)]

Unique HHIDs in df1: 2897
Unique HHIDs in df2: 2886
Unique HHIDs in df3: 2897
Unique HHIDs in df4: 2487
Unique HHIDs in df5: 2897
Unique HHIDs in df6: 2897
Common HHIDs across: 2479
df1: 2479 rows after filtering
df2: 7507 rows after filtering
df3: 2479 rows after filtering
df4: 5155 rows after filtering
df5: 2479 rows after filtering
df6: 2479 rows after filtering


In [788]:
print(df5.columns)
print(df6.columns)

Index(['HHID', 'Building_serial', 'Building_value'], dtype='object')
Index(['HHID', 'Land_serial', 'Land_value'], dtype='object')


In [789]:
# Merging all datasets
from functools import reduce

dfs = [df1, df2, df3, df4, df5, df6]

merged_df = reduce(lambda left, right: pd.merge(left, right, on='HHID', how='inner'), dfs)

merged_df.to_stata("merged.dta", write_index=False)

unique_hhids = set(merged_df['HHID'].dropna())
print(f"Unique HHIDs: {len(unique_hhids)}")

# Real Estate Value

merged_df['Real Estate'] = merged_df['Building_value'] + merged_df['Land_value']

merged_df.to_stata("merged1.dta", write_index=False)

merged_df1 = merged_df.copy()

merged_df1.columns

merged_df1.drop(['Building_value', 'Land_value'
                ], axis=1, inplace=True)

merged_df1.to_stata("merged1.dta", write_index=False)

merged_df1.columns

# Check for duplicates
group_cols = ['HHID', 'Credit_Agency', 'Fin_Asset_Serial', 'Share_Asset_serial']
dup_check = merged_df1.duplicated(subset=group_cols)
print(f"Number of duplicates: {dup_check.sum()}")

unique_count = merged_df1.loc[merged_df1['Fin_Asset_Serial'] == 37, 'HHID'].nunique()
print(f"Unique HHIDs with Fin_Asset_Serial = 37: {unique_count}")

unique_count = merged_df1.loc[merged_df1['Share_Asset_serial'] == 8, 'HHID'].nunique()
print(f"Unique HHIDs with Share_Asset_serial = 5: {unique_count}")

unique_count = merged_df1.loc[merged_df1['Credit_Agency'] == 999, 'HHID'].nunique()
print(f"Unique HHIDs with Credit_Agency = 999: {unique_count}")


Unique HHIDs: 2479
Number of duplicates: 0
Unique HHIDs with Fin_Asset_Serial = 37: 2479
Unique HHIDs with Share_Asset_serial = 5: 2479
Unique HHIDs with Credit_Agency = 999: 2479


C:\Users\siddu\AppData\Local\Temp\ipykernel_39844\1237505174.py:17: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  merged_df.to_stata("merged1.dta", write_index=False)
C:\Users\siddu\AppData\Local\Temp\ipykernel_39844\1237505174.py:26: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  merged_df1.to_stata("merged1.dta", write_index=False)


In [790]:
cols = ['HHID', 'State', 'MCE', 'Fin_Asset_Serial', 'Share_Asset_serial',
        'Credit_Agency', 'Share_Asset_Value', 'Fin_Asset_Value',
        'Liability_value', 'Real Estate']

# Create summary dataframe
summary = pd.DataFrame({
    'Column': cols,
    'Missing_Values': [merged_df1[col].isna().sum() for col in cols],
    'Zero_Values': [(merged_df1[col] == 0).sum() for col in cols]
})

print(summary)


# Unique HHIDs

unique_hhids = set(merged_df1['HHID'].dropna())
print(f"Unique HHIDs: {len(unique_hhids)}")

merged_df1.to_stata("merged22.dta", write_index=False)


               Column  Missing_Values  Zero_Values
0                HHID               0            0
1               State               0            0
2                 MCE               0            0
3    Fin_Asset_Serial               0            0
4  Share_Asset_serial               0            0
5       Credit_Agency               0            0
6   Share_Asset_Value               0           23
7     Fin_Asset_Value               0           28
8     Liability_value               0          325
9         Real Estate               0            0
Unique HHIDs: 2479


C:\Users\siddu\AppData\Local\Temp\ipykernel_39844\3725251504.py:20: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  merged_df1.to_stata("merged22.dta", write_index=False)


In [791]:
merged_df3 = merged_df1.copy()

In [792]:
# Create the new column
merged_df3['Fin_Asset'] = merged_df3['Fin_Asset_Serial'].map(Fin_Asset_map)
merged_df3['Share_Asset'] = merged_df3['Share_Asset_serial'].map(Share_Asset_Map)
merged_df3['Liability'] = merged_df3['Credit_Agency'].map(Credit_Agency_Map)

In [793]:
# Unique HHIDs

unique_hhids = set(merged_df3['HHID'].dropna())
print(f"Unique HHIDs: {len(unique_hhids)}")

Unique HHIDs: 2479


In [794]:
# HHIDs satisfying each condition
hhid_fin = set(merged_df3.loc[merged_df3['Fin_Asset_Serial'] == 37, 'HHID'])
hhid_share = set(merged_df3.loc[merged_df3['Share_Asset_serial'] == 8, 'HHID'])
hhid_credit = set(merged_df3.loc[merged_df3['Credit_Agency'] == 999, 'HHID'])

# Intersection: HHIDs present in all three
common_hhids = hhid_fin & hhid_share & hhid_credit

# Keep only those rows
filtered_df = merged_df3[merged_df3['HHID'].isin(common_hhids)].copy()

# Print how many unique HHIDs remain
print(f"Unique HHIDs present in all three groups: {len(common_hhids)}")

#filtered_df.drop(['Fin_Asset', 'Share_Asset', 'Liability'], axis=1, inplace=True)

#filtered_df.to_stata("final(1).dta", write_index=False)

Unique HHIDs present in all three groups: 2479


In [795]:
# Sanity checks if grouping worked correctly

print(sorted(filtered_df['Fin_Asset_Serial'].unique()))
#print(filtered_df['Fin_Asset_Serial'].value_counts())

print(sorted(filtered_df['Share_Asset_serial'].unique()))
#print(filtered_df['Share_Asset_serial'].value_counts())

print(sorted(filtered_df['Credit_Agency'].unique()))
#print(filtered_df['Credit_Agency'].value_counts())

[31, 32, 34, 35, 36, 37]
[8]
[91.0, 92.0, 999.0]


In [796]:
df_grouped1 = filtered_df.copy()

In [797]:
# Step 1: Filter the relevant rows satisfying your condition
wealth_rows = df_grouped1[
    (df_grouped1['Credit_Agency'] == 999) &
    (df_grouped1['Fin_Asset_Serial'] == 37) &
    (df_grouped1['Share_Asset_serial'] == 8)
].copy()

# Step 2: Compute wealth for these rows
wealth_rows['Gross wealth'] = (
    wealth_rows['Fin_Asset_Value'] +
    wealth_rows['Share_Asset_Value'] +
    wealth_rows['Real Estate']
)

wealth_rows['Total Financial Assets'] = (
    wealth_rows['Fin_Asset_Value']+
    wealth_rows['Share_Asset_Value']
)

wealth_rows['Total Institutional liabilities'] = wealth_rows['Liability_value']

# Step 3: Keep only HHID and wealth columns
wealth_per_hhid = wealth_rows[['HHID', 'Gross wealth', 'Total Financial Assets', 'Total Institutional liabilities']]

# Step 4: Merge wealth back to the full dataset based on HHID
df_grouped1 = df_grouped1.merge(wealth_per_hhid, on='HHID', how='left')

# Step 5: View result
print(df_grouped1[['HHID', 'Gross wealth']].drop_duplicates().head())

df_grouped1.to_stata("final(25).dta", write_index=False)

       HHID  Gross wealth
0   3104731       72580.0
4   3107711        8650.0
8   3107712        5370.0
12  3109751      104679.0
18  3110751      409361.0


C:\Users\siddu\AppData\Local\Temp\ipykernel_39844\2515997013.py:31: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate
    Gross wealth   ->   Gross_wealth
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutional liabilities   ->   Total_Institutional_liabilities

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  df_grouped1.to_stata("final(25).dta", write_index=False)


In [798]:
df_grouped2 =  df_grouped1.copy()

In [799]:
import numpy as np

# Initialize columns with NaN
df_grouped2['Asset_serial'] = np.nan
df_grouped2['Asset_Value'] = np.nan
df_grouped2['Asset_Name'] = np.nan

group_cols = ['HHID', 'Credit_Agency', 'Fin_Asset_Serial', 'Share_Asset_serial']

df_grouped2['HHID'] = df_grouped2['HHID'].astype(str)

#df_grouped2.to_stata("final(26).dta", write_index=False)


In [800]:
# Empty list to store new rows
rows = []

# Loop over each unique group
for _, group in df_grouped2.groupby(group_cols):

    # Get first row for copying other columns
    first_row = group.iloc[0].to_dict()

    # 1. Financial Asset Row
    if not np.isnan(first_row['Fin_Asset_Value']):
        fa_row = first_row.copy()
        fa_row['Asset_serial'] = first_row['Fin_Asset_Serial']
        fa_row['Asset_Value'] = first_row['Fin_Asset_Value']
        fa_row['Asset_Name'] = first_row['Fin_Asset']
        rows.append(fa_row)

    # 2. Share Asset Row
    if not np.isnan(first_row['Share_Asset_Value']):
        sa_row = first_row.copy()
        sa_row['Asset_serial'] = 200 + first_row['Share_Asset_serial']  # As you specified
        sa_row['Asset_Value'] = first_row['Share_Asset_Value']
        sa_row['Asset_Name'] = first_row['Share_Asset']
        rows.append(sa_row)

    # 3. Real Estate Row
    if not np.isnan(first_row['Real Estate']):
        re_row = first_row.copy()
        re_row['Asset_serial'] = 300
        re_row['Asset_Value'] = first_row['Real Estate']
        re_row['Asset_Name'] = 'Real Estate'
        rows.append(re_row)

    # 4. Gross wealth Row
    if not np.isnan(first_row['Gross wealth']):
        w_row = first_row.copy()
        w_row['Asset_serial'] = 1000
        w_row['Asset_Value'] = first_row['Gross wealth']
        w_row['Asset_Name'] = 'Gross wealth'
        rows.append(w_row)

    # 6. Total Financial Assets Row
    if not np.isnan(first_row['Total Financial Assets']):
        tfa_row = first_row.copy()
        tfa_row['Asset_serial'] = 1500
        tfa_row['Asset_Value'] = first_row['Total Financial Assets']
        tfa_row['Asset_Name'] = 'Total Financial Assets'
        rows.append(tfa_row)   

# Convert to DataFrame
asset_df = pd.DataFrame(rows)

# Optional: Arrange columns nicely
cols_order = group_cols + ['Asset_serial', 'Asset_Value'] + [col for col in df_grouped2.columns if col not in group_cols + ['Asset_serial', 'Asset_Value']]
asset_df = asset_df[cols_order]

# Check result
print(asset_df.head())



# Renaming Liabilities

asset_df = asset_df.rename(columns={
    'Credit_Agency': 'Liability_serial',
    'Liability': 'Liability_name'
})

asset_df.to_stata("final(28).dta", write_index=False)


      HHID  Credit_Agency  Fin_Asset_Serial  Share_Asset_serial  Asset_serial  \
0  3104731           91.0                31                   8            31   
1  3104731           91.0                31                   8           208   
2  3104731           91.0                31                   8           300   
3  3104731           91.0                31                   8          1000   
4  3104731           91.0                31                   8          1500   

   Asset_Value  State     MCE  Fin_Asset_Value  Share_Asset_Value  ...  \
0        380.0      4  1400.0            380.0             1200.0  ...   
1       1200.0      4  1400.0            380.0             1200.0  ...   
2      71000.0      4  1400.0            380.0             1200.0  ...   
3      72580.0      4  1400.0            380.0             1200.0  ...   
4       1580.0      4  1400.0            380.0             1200.0  ...   

   Building_serial  Land_serial  Real Estate  Fin_Asset  \
0        

C:\Users\siddu\AppData\Local\Temp\ipykernel_39844\347409114.py:69: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate
    Gross wealth   ->   Gross_wealth
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutional liabilities   ->   Total_Institutional_liabilities

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  asset_df.to_stata("final(28).dta", write_index=False)


In [801]:
group_cols = ['HHID', 'Liability_serial', 'Fin_Asset_Serial', 'Share_Asset_serial']
group_cols1 = ['HHID', 'Liability_serial', 'Fin_Asset_Serial', 'Share_Asset_serial', 'Asset_serial']

# Count unique combinations
num_subsets = asset_df[group_cols].drop_duplicates().shape[0]

print(f"Number of unique subsets: {num_subsets}")

# Check for duplicates
dup_check = asset_df.duplicated(subset=group_cols)
print(f"Number of duplicates: {dup_check.sum()}")

# Check for duplicates
dup_check = asset_df.duplicated(subset=group_cols1)
print(f"Number of duplicates: {dup_check.sum()}")

Number of unique subsets: 15856
Number of duplicates: 63424
Number of duplicates: 0


In [802]:
asset_df.dtypes

HHID                                object
Liability_serial                   float64
Fin_Asset_Serial                     int64
Share_Asset_serial                   int64
Asset_serial                         int64
Asset_Value                        float64
State                                int64
MCE                                float64
Fin_Asset_Value                    float64
Share_Asset_Value                  float64
Liability_value                    float64
Building_serial                      int64
Land_serial                          int64
Real Estate                          int64
Fin_Asset                           object
Share_Asset                         object
Liability_name                      object
Gross wealth                       float64
Total Financial Assets             float64
Total Institutional liabilities    float64
Asset_Name                          object
dtype: object

In [803]:
# Check for missing values

# Check how many missing in each column
print(asset_df[['Asset_serial', 'Asset_Value', 'Asset_Name']].isna().sum())

# Optionally, view rows with missing in any of them
missing_rows = asset_df[
    asset_df[['Asset_serial', 'Asset_Value', 'Asset_Name']].isna().any(axis=1)
]

print(f"\nTotal rows with missing values: {len(missing_rows)}")
print(missing_rows.head())


Asset_serial    0
Asset_Value     0
Asset_Name      0
dtype: int64

Total rows with missing values: 0
Empty DataFrame
Columns: [HHID, Liability_serial, Fin_Asset_Serial, Share_Asset_serial, Asset_serial, Asset_Value, State, MCE, Fin_Asset_Value, Share_Asset_Value, Liability_value, Building_serial, Land_serial, Real Estate, Fin_Asset, Share_Asset, Liability_name, Gross wealth, Total Financial Assets, Total Institutional liabilities, Asset_Name]
Index: []

[0 rows x 21 columns]


In [804]:
final_df = asset_df.copy()

In [805]:
final_df.columns

Index(['HHID', 'Liability_serial', 'Fin_Asset_Serial', 'Share_Asset_serial',
       'Asset_serial', 'Asset_Value', 'State', 'MCE', 'Fin_Asset_Value',
       'Share_Asset_Value', 'Liability_value', 'Building_serial',
       'Land_serial', 'Real Estate', 'Fin_Asset', 'Share_Asset',
       'Liability_name', 'Gross wealth', 'Total Financial Assets',
       'Total Institutional liabilities', 'Asset_Name'],
      dtype='object')

In [806]:
final_df.drop(['Fin_Asset_Serial', 
               'Share_Asset_serial', 
               'Fin_Asset_Value',
                'Share_Asset_Value', 
                'Fin_Asset',
                'Share_Asset',               
                ], axis=1, inplace=True)

final_df.to_stata("final(1992).dta", write_index=False)

final_df = final_df[['HHID', 'State', 'MCE','Gross wealth',  
                    'Liability_serial', 'Liability_name','Liability_value',
                    'Asset_serial', 'Asset_Name', 'Asset_Value',
                    'Real Estate', 'Total Financial Assets', 'Total Institutional liabilities', 
                    ]].copy()

final_df.to_stata("final(1992).dta", write_index=False)

final_df1 = final_df[['HHID', 'State', 'MCE', 'Gross wealth',  
                    'Liability_serial', 'Liability_name','Liability_value',
                    'Asset_serial', 'Asset_Name', 'Asset_Value'
                    ]].copy()

final_df1.to_stata("final(1992)_1.dta", write_index=False)

final_df1['HHID'].nunique()

C:\Users\siddu\AppData\Local\Temp\ipykernel_39844\3300125772.py:9: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate
    Gross wealth   ->   Gross_wealth
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutional liabilities   ->   Total_Institutional_liabilities

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  final_df.to_stata("final(1992).dta", write_index=False)
C:\Users\siddu\AppData\Local\Temp\ipykernel_39844\3300125772.py:17: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Gross wealth   ->   Gross_wealth
    Real Estate   ->   Real_Estate
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutiona

2479

In [807]:
print(sorted(final_df1['Asset_serial'].unique()))

[31, 32, 34, 35, 36, 37, 208, 300, 1000, 1500]


In [808]:
group_cols1 = ['HHID', 'Liability_serial', 'Asset_serial']
final_df2 = final_df1.drop_duplicates(subset=group_cols1, keep='first').reset_index(drop=True)
final_df2.to_stata("final(1992)_2.dta", write_index=False)

C:\Users\siddu\AppData\Local\Temp\ipykernel_39844\2175703513.py:3: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Gross wealth   ->   Gross_wealth

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  final_df2.to_stata("final(1992)_2.dta", write_index=False)


In [809]:
# Check for repetition

# Check for uniqueness 

duplicates = final_df2.duplicated(subset=['Asset_serial', 'HHID', 'Liability_serial'], keep=False)

# Check if there are any duplicates
if duplicates.any():
    print("Some (Asset_serial, HHID) combinations are repeated:")
    print(final_df2.loc[duplicates, ['Asset_serial', 'HHID']].sort_values(['Asset_serial', 'HHID']))
else:
    print(" All HHIDs are unique within each Asset_serial category.")

# Check if there are any duplicates
if duplicates.any():
    print("Some (Liability_serial, HHID) combinations are repeated:")
    print(final_df2.loc[duplicates, ['Liability_serial', 'HHID']].sort_values(['Liability_serial', 'HHID']))
else:
    print(" All HHIDs are unique within each Liability_serial category.")


 All HHIDs are unique within each Asset_serial category.
 All HHIDs are unique within each Liability_serial category.


In [810]:
final_df2['HHID'].nunique()

2479

# Add State names corresponding to their codes

In [811]:
# final_df2 (Merge D&D, D&N)
final_df2.loc[final_df2['State'].isin([27,33]), 'State'] = 27
final_df2['State'].value_counts()

# State Map

State_map = {
    2.0: 'Andhra Pradesh',
    3.0: 'Assam',
    4.0: 'Bihar',
    5.0: 'Gujarat',
    6.0: 'Haryana',
    7.0: 'Himachal Pradesh',
    8.0: 'Jammu & Kashmir',
    9.0: 'Karnataka',
    10.0: 'Kerala',
    11.0: 'Madhya Pradesh',
    12.0: 'Maharashtra',
    13.0: 'Manipur',
    14.0: 'Meghalaya',
    15.0: 'Nagaland',
    16.0: 'Odisha',
    17.0: 'Punjab',
    18.0: 'Rajasthan',
    19.0: 'Sikkim',
    20.0: 'Tamilnadu',
    21.0: 'Tripura',
    22.0: 'Uttar Pradesh',
    23.0: 'West Bengal',
    24.0: 'Andaman & Nicobar',
    25.0: 'Arunachal Pradesh',
    26.0: 'Chandigarh',
    27.0: 'Daman and Diu and Dadra and Nagar Haveli',
    28.0: 'Delhi',
    29.0: 'Goa',
    30.0: 'Lakshadweep',
    31.0: 'Mizoram',
    32.0: 'Puducherry'
}

final_df3 = final_df2.copy()

final_df3['State_name'] = final_df3['State'].map(State_map)

final_df3.to_stata('1992_final.dta', write_index=False)

C:\Users\siddu\AppData\Local\Temp\ipykernel_39844\932734623.py:45: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Gross wealth   ->   Gross_wealth

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  final_df3.to_stata('1992_final.dta', write_index=False)
